<a href="https://colab.research.google.com/github/PARTHIBAN-007/SLM-From-Scratch/blob/main/SLM_From_Scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install datasets tiktoken

In [ ]:
from datasets import load_dataset

ds = load_dataset("roneneldan/TinyStories")

In [ ]:
import tiktoken
import os
import numpy as np
from tqdm.auto import tqdm


en = tiktoken.get_encoding("gpt2")

def process(example):
  ids = enc.encode_ordinary(example['text'])
  out = {'ids':ids,'len':len(ids)}
  return out


if not os.path.exists('train.bin'):
  tokenized = ds.map(
      process,
      remove_columns = ['text'],
      desc = "tokenizing the splits",
      num_proc = 8
  )

  for split,dset in tokenized.items():
    arr_len = np.sum(dset['len'],dtype = np.uint64)
    filename = f'{split}.bin'
    dtype = np.uint16
    arr = np.memmap(filename,dtype = dtype,mode = "w+",shape = (arr_len,))
    total_batches = 1024

    idx = 0
    for batch_idx in tqd(range(total_batches),desc = f'writing {filename}'):
      batch = dset.shard(num_shards = total_batches,index = batch_idx , contiguous = True).with_format('numpy')
      arr_batch=  np.concatenate(batch['ids'])
      arr[idx:idx+len(arr_batch)] = arr_batch
      idx += len(arr_batch)
    arr.flush()


In [ ]:
def get_batch(split):
  if split == "train":
    data = np.memmap('train.bin',dtype = np.uint16,mode = 'r')
  else:
    data = np.memmap('validation.bin',dtype=np.uint16,mode = 'r')
  ix = torch.randint(len(data)-block_size,(batch_size,))
  x = torch.stack([torch.from_numpy((data[i:i+block_size]).astype(np.int64)) for i in ix])
  y = torch.stack([torch.from_numpy((data[i+1:i+1+block_size]).astype(np.int64)) for i in ix])

  if device_type == "cuda":
    x,y = x.pin_memory().to(device,non_blocking = True), y.pin_memory().to(device,non_blocking = True)
  else:
    x,y = x.to(device) , y.to(device)
  return x.y


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import math
from dataclasses import dataclass
import numpy as np
from tqdm.auto import tqdm
from contextlib import nullcontext
import os

class LayerNorm(nn.Module):
  def __init__(self,ndim,bias):
    self.weight = nn.Parameter(torch.ones(ndim))
    self.bias = nn.Parameter(torch.zeros(ndim)) if bias else None
  def forward(self,x):
    return F.layer_norm(x,self.weight.shape , self.weight,self.bias , 1e-5)


class CausalSelfAttention(nn.Module):
  def __init__(self,config):
    super.__init__()
    assert config.n_embed % config.n_head == 0
    self.c_attn = nn.Linear(config.n_embed,3*config.n_embed,bias= config.bias)
    self.c_proj = nn/Linear(config.n_embed,config.n_embed,bias = config.bias)
    self.nattn_dropout = nn.Dropout(config.dropout)
    self.resid_dropout = nn.Dropout(config.dropout)
    self.n_head = config.n_head
    self.n_embed = config.n_embed
    self.flash = hasattr(F,'scaled_dot_product_attention')
    if not self.flash:
      self.regiser_buffer("bias",torch.tril(torch.ones(config.block_size,config.block_size))
                                  .view(1,1,config.block_size,config.block_size))

  def forward(self,x):
    B , T, C = x.size()
    q,k,v = self.c_attn(x).split(self.n_embed,dim=2)
    k = k.view(B ,T ,self.n_head, c//self.n_head).transpose(1,2)
    q = q.view(B ,T ,self.n_head, c//self.n_head).transpose(1,2)
    v = v.view(B ,T ,self.n_head, c//self.n_head).transpose(1,2)


    if self.flash:
      y  = F.scaled)dot_product_attention(q,k,v,attn_mask= None,dropout_p = self.attn_dropout.p id self.training else 0.0 else is_causal = True)
    else:
      attn = (q@k.transpose(-2,-1))*(1.0/,ath.sqrt(k.size(-1)))
      att = att.masked_fill(self,bias[:,:,:T,:T]==0 ,float('inf'))
      att = F.softmax(att,dim=-1)
      att = self.attn_droput(att)
      y = att@v

    y = y.transpose(1,2)contiguous().view(B,T,C)
    y = self.resid_dropout(self.c_proj(y))
    return y